In [1]:
state = 'IA' 
year = 2020
obj_type = 'average_Polsby_Popper'
starting_deviation = 0.01 

In [2]:
import sys, os
src_path = os.path.abspath(os.path.join('..', '..', 'src'))
sys.path.append(src_path)

In [3]:
filepath = '../../dat/' + str(year) + '/'
filename = state + '_county.json'
filename2 = state + '_county.shp'

In [4]:
from read import read_graph_from_json

G = read_graph_from_json(state, filepath + filename, year=year)
print(f"The state of {state} has {G._k} districts.")
G._ideal_population = sum(G.nodes[i]['TOTPOP'] for i in G.nodes) / G._k

The state of IA has 4 districts.


In [5]:
#import warm starts
sys.path.append(os.path.abspath('../heuristic'))

from IA_plans_2020 import plans
print(f"Loaded {len(plans)} plans from file.")
warm_starts = plans

Loaded 18 plans from file.


In [6]:
from pareto import filter_and_sort_pareto
from metrics import scores

plans_scores = [scores(G, plan, G._ideal_population, obj_type) for plan in warm_starts]
_,_,nondominated_warm_starts_plans = filter_and_sort_pareto(plans=warm_starts, upper_bounds=plans_scores, obj_type=obj_type)
print(f"Selected {len(nondominated_warm_starts_plans)} nondominated warm start plans")

Selected 10 nondominated warm start plans


In [ ]:
from epsilon_constraint import epsilon_constraint_method

(plans, obj_bounds, deviations) = epsilon_constraint_method(
            G,                 
            obj_type,          
            contiguity = 'lcut',                                             # {'lcut', 'scf', 'shir'} 
            cutoff=None,       
            verbose= True,
            warm_start_mode = 'user',                                        # {'None', 'user', 'refinement'}
            warm_starts=warm_starts,                                         # if you have user define warm starts else it is None
            starting_deviation=starting_deviation, 
            time_limit=7200, 
            sizes=None,      
            max_B=True,                                                      # If symmetry_breaking is 'orbitope' or you have warm_start, max_B should be True   
            symmetry_breaking='orbitope',                                    # {None, 'orbitope', 'rsum'} 
            state=state,
            year=year
        )

Initially, L = 789617 and U = 805568 and k = 4.

****************************************
Trying deviation = 7975.922500000001
****************************************
Using user-provided warm starts.
Selected warm_start = [[1, 2, 8, 14, 20, 21, 22, 27, 41, 59, 60, 65, 68, 71, 76, 78, 79, 84, 92], [3, 10, 12, 18, 24, 25, 26, 28, 38, 39, 44, 46, 47, 48, 54, 61, 67, 83, 86, 94, 95, 96, 97, 98], [0, 4, 9, 16, 17, 19, 23, 30, 31, 32, 33, 35, 36, 37, 40, 42, 43, 49, 50, 52, 53, 55, 57, 58, 62, 63, 64, 66, 70, 73, 74, 81, 89, 91, 93], [5, 6, 7, 11, 13, 15, 29, 34, 45, 51, 56, 69, 72, 75, 77, 80, 82, 85, 87, 88, 90]]
Objective value: 0.47624911339613396
Deviation: 663.25

****************************************
Running labeling model!
****************************************
L = 789617 and U = 805568
Set parameter Username
Set parameter LicenseID to value 2608266
Academic license - for non-commercial use only - expires 2026-01-09
sizes =  [1, 1, 1, 1]
Solving the max B problem (as MIP) for u

H 9622  4434                       0.4976842    0.99104  99.1%   457  511s
H 9770  4434                       0.4976881    0.99104  99.1%   456  511s
  9909  4567    0.51391   44  537    0.49769    0.99104  99.1%   458  527s
 10324  4721 infeasible   34         0.49769    0.98991  98.9%   459  544s
 10719  4931    0.62261   28  518    0.49769    0.96350  93.6%   458  561s
 11185  5206    0.80863   25  606    0.49769    0.96317  93.5%   459  578s
 11760  5475    0.92893   25  650    0.49769    0.93412  87.7%   455  597s
 12362  5732    0.65241   32  637    0.49769    0.93296  87.5%   453  617s
 12963  5944    0.65361   27  521    0.49769    0.93277  87.4%   451  637s
 13453  6149    0.55984   30  553    0.49769    0.93247  87.4%   451  652s
 14007  6346    0.65495   27  616    0.49769    0.93190  87.2%   450  668s
 14597  6606 infeasible   19         0.49769    0.93176  87.2%   448  685s
 15278  6780    0.49958   27  413    0.49769    0.93171  87.2%   448  706s
 15808  6981    0.86441  

 202716 64522    0.62008   32  581    0.49769    0.74358  49.4%   383 6137s
 204858 65006    0.52949   29  636    0.49769    0.74327  49.3%   382 6204s
 206545 65006    0.59134   31  595    0.49769    0.74304  49.3%   382 6205s
 207213 65474     cutoff   35         0.49769    0.74166  49.0%   382 6263s
 209137 66023    0.71557   24  565    0.49769    0.73772  48.2%   381 6324s
 210361 66023    0.53028   32  526    0.49769    0.73728  48.1%   381 6325s
 211387 66663    0.52850   36  551    0.49769    0.71773  44.2%   381 6385s
 213674 67367    0.49922   36  634    0.49769    0.71692  44.0%   381 6447s
 216139 68041     cutoff   40         0.49769    0.71656  44.0%   380 6505s
 218640 68542    0.56139   32  601    0.49769    0.71629  43.9%   379 6566s
 220742 69104    0.56082   30  519    0.49769    0.71607  43.9%   379 6624s
 223030 69518    0.59158   35  560    0.49769    0.71588  43.8%   379 6674s
 224789 69518    0.62259   34  557    0.49769    0.71583  43.8%   378 6675s
 225132 7010

   732   365    0.99158   13  520    0.47625    0.99158   108%   822  276s
   740   378    0.99158   14  534    0.47625    0.99158   108%   872  282s
   756   393    0.99158   15  536    0.47625    0.99158   108%   924  289s
   776   456    0.99158   16  532    0.47625    0.99158   108%   964  296s
   852   476    0.87307   18  561    0.47625    0.99158   108%   928  302s
   901   499    0.99158   19  538    0.47625    0.99158   108%   905  308s
   943   509    0.99158   20  537    0.47625    0.99158   108%   892  312s
   981   535    0.99158   21  550    0.47625    0.99158   108%   877  316s
  1070   561 infeasible   23         0.47625    0.99158   108%   849  323s
  1111   579    0.81057   24  568    0.47625    0.99158   108%   846  326s
  1198   582    0.80984   25  565    0.47625    0.99158   108%   815  330s
  1287   583 infeasible   26         0.47625    0.99158   108%   787  337s
  1334   577    0.74787   27  540    0.47625    0.99158   108%   771  340s
  1384   583    0.74820  

 81711 33409    0.68370   34  560    0.47625    0.74705  56.9%   425 3317s
 84467 34178    0.59243   32  600    0.47625    0.74689  56.8%   422 3391s
 87063 34925    0.74357   27  544    0.47625    0.74671  56.8%   420 3460s
 89490 35859    0.65434   33  452    0.47625    0.74651  56.7%   417 3530s
 92526 36615 infeasible   34         0.47625    0.74627  56.7%   413 3591s
 95578 37144    0.71624   32  623    0.47625    0.74595  56.6%   409 3654s
 98378 37795    0.62450   25  578    0.47625    0.74550  56.5%   407 3719s
 101334 38525 infeasible   41         0.47625    0.74481  56.4%   404 3789s
 102803 38525    0.53018   31  516    0.47625    0.74476  56.4%   403 3790s
 104043 39272 infeasible   25         0.47625    0.74360  56.1%   402 3857s
 106214 39956    0.59189   37  739    0.47625    0.71835  50.8%   402 3927s
 108210 40530    0.56035   40  331    0.47625    0.71779  50.7%   401 3991s
 110373 41309    0.68423   35  448    0.47625    0.71728  50.6%   401 4053s
 112771 42081 infea

...

 232420 108087    0.39018   50  379    0.38001    0.74839  96.9%   371 5812s
 234381 109323 infeasible   32         0.38001    0.74831  96.9%   371 5856s
 236953 110406    0.68391   36  665    0.38001    0.74821  96.9%   370 5898s
 239166 111558 infeasible   29         0.38001    0.74815  96.9%   370 5942s
 241213 112787    0.56211   30  460    0.38001    0.74808  96.9%   370 5985s
 243595 113906 infeasible   45         0.38001    0.74801  96.8%   370 6022s
 246002 114849    0.74499   37  684    0.38001    0.74797  96.8%   369 6061s
 247810 116068    0.74581   30  631    0.38001    0.74791  96.8%   370 6104s
 250162 117277    0.41211   73  290    0.38001    0.74786  96.8%   369 6146s
 252428 118151    0.74494   37  584    0.38001    0.74782  96.8%   369 6182s
 254094 119471 infeasible   32         0.38001    0.74781  96.8%   369 6228s
 256983 120551    0.62347   33  509    0.38001    0.74776  96.8%   368 6281s
 259251 121606    0.74669   31  577    0.38001    0.74770  96.8%   368 6330s

   846   480    0.99457   23  534    0.30066    0.99457   231%   834  218s
   888   500    0.99457   24  556    0.30066    0.99457   231%   822  221s
   934   520    0.99457   25  582    0.30066    0.99457   231%   808  225s
  1012   559    0.99457   26  587    0.30066    0.99457   231%   794  234s
  1054   577 infeasible   27         0.30066    0.99457   231%   796  237s
  1100   579    0.87211   28  563    0.30066    0.99457   231%   780  241s
  1179   611    0.81082   30  569    0.30066    0.99457   231%   764  246s
  1268   650    0.82375   32  635    0.30066    0.99457   231%   743  252s
  1352   657    0.79290   34  633    0.30066    0.99457   231%   729  256s
  1455   655    0.99457   22  546    0.30066    0.99457   231%   718  260s
  1500   679    0.99439   24  555    0.30066    0.99457   231%   713  268s
  1549   704    0.74921   26  579    0.30066    0.99457   231%   714  272s
  1616   714    0.74921   27  576    0.30066    0.99457   231%   698  275s
  1680   725    0.74829  

 110308 60993 infeasible   42         0.30066    0.80830   169%   335 2687s
 112651 62377    0.34328   50  457    0.30066    0.80821   169%   334 2735s
 115207 63898    0.68367   38  431    0.30066    0.80810   169%   333 2779s
 117842 65151    0.65139   41  445    0.30066    0.80803   169%   332 2824s
 120343 66106    0.43665   43  586    0.30066    0.80793   169%   331 2866s
 122326 67342    0.34327   44  505    0.30066    0.80782   169%   330 2909s
 124917 68576    0.53000   36  317    0.30066    0.80769   169%   329 2946s
 127281 70201 infeasible   34         0.30066    0.80756   169%   329 2990s
 130270 71922    0.36718   85  352    0.30066    0.80746   169%   327 3038s
 133133 73131    0.40591   57  258    0.30066    0.80735   169%   325 3087s
 135494 74421    0.37321   48  514    0.30066    0.80715   168%   325 3132s
 137949 75934    0.46698   45  377    0.30066    0.80711   168%   324 3179s
 140224 75934    0.46798   36  528    0.30066    0.80696   168%   323 3180s
 140770 7711

 341901 195799    0.46679   44  595    0.30066    0.74674   148%   299 6836s
 344153 196764    0.68445   32  561    0.30066    0.74672   148%   299 6877s
 346165 198018 infeasible   35         0.30066    0.74671   148%   299 6919s
 348306 199444 infeasible   36         0.30066    0.74669   148%   299 6956s
 350673 200408    0.37362   61  505    0.30066    0.74667   148%   299 6992s
 352479 201558    0.68338   34  585    0.30066    0.74665   148%   299 7034s
 354456 202690    0.30926   54  421    0.30066    0.74664   148%   299 7074s
 356500 203660    0.49840   37  438    0.30066    0.74662   148%   299 7112s
 358079 204841    0.40483   45  535    0.30066    0.74660   148%   299 7149s
 360148 206306    0.40466   54  318    0.30066    0.74658   148%   299 7190s
 362563 206563    0.62329   28  529    0.30066    0.74657   148%   299 7200s

Cutting planes:
  Gomory: 3
  Clique: 1
  Flow cover: 19
  Zero half: 11
  RLT: 16

Explored 362980 nodes (108558041 simplex iterations) in 7204.21 seco

  1857   806 infeasible   42         0.27355    0.98632   261%   734  363s
  1950   841    0.98019   21  569    0.27355    0.98632   261%   712  367s
  2052   879    0.91740   23  585    0.27355    0.98632   261%   696  372s
  2132   924    0.91816   23  597    0.27355    0.98632   261%   689  376s
  2299  1044    0.77907   29  586    0.27355    0.98632   261%   670  380s
  2383  1118    0.74785   32  585    0.27355    0.98632   261%   663  385s
  2477  1190    0.74582   35  563    0.27355    0.98632   261%   652  392s
  2586  1255    0.74676   38  589    0.27355    0.98632   261%   641  398s
  2701  1336    0.91736   25  556    0.27355    0.98632   261%   632  403s
  2819  1417    0.85016   30  627    0.27355    0.98632   261%   619  408s
  2928  1524    0.72936   33  533    0.27355    0.98632   261%   610  414s
  3074  1615    0.51456   42  553    0.27355    0.98632   261%   596  417s
  3238  1697    0.46221   53  409    0.27355    0.98632   261%   580  421s
  3376  1777    0.97890  

In [ ]:
result = list(zip(plans, obj_bounds , deviations))

In [ ]:
epsilon = 1 / (2 * G._k)
min_deviation = min(round(r[2], 1) for r in result)

if min_deviation < epsilon:
    no_solution_region = None
else:
    no_solution_region = [0, min_deviation]

if no_solution_region is not None:
    print(f"No feasible solution was found within the region: {no_solution_region}")

In [ ]:
from pareto import plot_pareto_frontiers

plot_pareto_frontiers(
                G,
                method='epsilon_constraint_method',
                plans=None,                                   #if method ='epsilon_constraint_method' is None 
                obj_types=obj_type,                               
                ideal_population=G._ideal_population,
                state=state,
                filepath=filepath,
                filename2=filename2,
                no_solution_region=no_solution_region,
                year=year,
                result=result                               #if method ='heuristic' is None 
             )

In [ ]:
from draw import draw_plan
from metrics import observed_deviation_persons, compute_obj

print(f"\n{'#' * 100}\nPareto maps for state {state}, objective {obj_type}\n{'#' * 100}\n")

format_obj = {
    'bottleneck_Polsby_Popper': lambda x: round(1 /x, 4),
    'cut_edges': lambda x: int(x)}
G._L = 0
G._U = G._k * G._ideal_population

for plan, obj_bound, dev in result:
    obs_dev = observed_deviation_persons(G, plan, G._ideal_population)
    obj = compute_obj(G, plan, obj_type)
    obj_val = format_obj.get(obj_type, lambda x: round(x, 4))(obj)
    deviation_percentage = round(100 * dev / G._ideal_population, 4)
    title = f"{round(obs_dev, 2)}-person deviation ({deviation_percentage}%), with {obj_val} {obj_type}"
    draw_plan(filepath, filename2, G, plan, title=title, year=year)